In [2]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

# Data Sources #

## London DataStore ##

https://data.london.gov.uk/dataset/london-fire-brigade-incident-records 

2 files listing all LFB callouts for 2017-2020 (inclusive) and 2013-2016 (inclusive) - 2016-2020 used for the project.

## Met Office Archives (CEDA Midas Open) ##

https://catalogue.ceda.ac.uk/uuid/dbd451271eb04662beade68da43546e1

1 file per year for hourly weather observations, with additional files per year for rain details and wind details for 2016-2019 (inclusive) - 12 files in total.

## Meteostat ##

https://github.com/meteostat Great resource (with python library and easy api) for historical weather data worldwide (initially was going to webscrape from the website - https://meteostat.net/en/station/03772) 1 File for 2020 - one observation (sun) missing from the dataset compared with met office records (and other observations to be feature engineered). Also used to backfill data missing from met office records.

## Earth System Research Laboratories ##

https://www.esrl.noaa.gov/gmd/grad/solcalc/calcdetails.html - excel spreadsheet to download to calculate sunrise/sunset times in London (needed for an 'is light' marker).
 
## Met Office Forecast (London) ##

https://www.metoffice.gov.uk/weather/forecast/gcpvj0v07#? - for current London forecast for the next 7 days (needed for current prediction).

## Met Office Actual (Heathrow) ##

https://www.metoffice.gov.uk/weather/observations/gcpsvg3nc - for current air pressure London (needed for current prediction).

# London Fire Brigade (LFB) Data

In [3]:
#LFB database for inspection (most recent csv file - 2017-2020)
LFB2017df = pd.read_csv('LFBIncident2017.csv', engine='python')
LFB2017df.describe(include='all')

,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,...,IncidentStationGround,FirstPumpArriving_AttendanceTime,FirstPumpArriving_DeployedFromStation,SecondPumpArriving_AttendanceTime,SecondPumpArriving_DeployedFromStation,NumStationsWithPumpsAttending,NumPumpsAttending,PumpCount,PumpHoursRoundUp,Notional Cost (£)
count,413008,413008,413008.000000,413008,413008.000000,413008,413008,129372,413008,413008,...,413008,389188.000000,389181,157869.000000,157865,410795.000000,410795.000000,411209.000000,411085.000000,411085.000000
unique,413008,1461,NaN,83348,NaN,3,10,21,9,284,...,102,NaN,105,NaN,102,NaN,NaN,NaN,NaN,NaN
top,008718-21012017,03/03/2018,NaN,18:38:33,NaN,False Alarm,AFA,Flooding,Dwelling,Purpose Built Flats/Maisonettes - 4 to 9 storeys,...,Soho,NaN,Soho,NaN,Soho,NaN,NaN,NaN,NaN,NaN
freq,1,835,NaN,19,NaN,208732,158276,28246,207191,53870,...,15037,NaN,14451,NaN,4442,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,2018.480664,NaN,13.476175,NaN,NaN,NaN,NaN,NaN,...,NaN,310.483193,NaN,388.523554,NaN,1.383006,1.565881,1.619994,1.374375,460.746160
std,NaN,NaN,1.108126,NaN,6.244883,NaN,NaN,NaN,NaN,NaN,...,NaN,132.087535,NaN,147.585822,NaN,0.719757,0.873267,1.661056,6.942830,2312.690899
min,NaN,NaN,2017.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,1.000000,NaN,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,326.000000
25%,NaN,NaN,2017.000000,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,228.000000,NaN,294.000000,NaN,1.000000,1.000000,1.000000,1.000000,328.000000
50%,NaN,NaN,2018.000000,NaN,14.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,292.000000,NaN,364.000000,NaN,1.000000,1.000000,1.000000,1.000000,339.000000
75%,NaN,NaN,2019.000000,NaN,19.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,369.000000,NaN,452.000000,NaN,2.000000,2.000000,2.000000,1.000000,346.000000


In [4]:
LFB2017df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413008 entries, 0 to 413007
Data columns (total 38 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   IncidentNumber                          413008 non-null  object 
 1   DateOfCall                              413008 non-null  object 
 2   CalYear                                 413008 non-null  int64  
 3   TimeOfCall                              413008 non-null  object 
 4   HourOfCall                              413008 non-null  int64  
 5   IncidentGroup                           413008 non-null  object 
 6   StopCodeDescription                     413008 non-null  object 
 7   SpecialServiceType                      129372 non-null  object 
 8   PropertyCategory                        413008 non-null  object 
 9   PropertyType                            413008 non-null  object 
 10  AddressQualifier                        4130

In [5]:
LFB2017df.head()

,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,...,IncidentStationGround,FirstPumpArriving_AttendanceTime,FirstPumpArriving_DeployedFromStation,SecondPumpArriving_AttendanceTime,SecondPumpArriving_DeployedFromStation,NumStationsWithPumpsAttending,NumPumpsAttending,PumpCount,PumpHoursRoundUp,Notional Cost (£)
0,000226-01012017,01/01/2017,2017,10:15:32,10,Special Service,Special Service,Flooding,Dwelling,Purpose Built Flats/Maisonettes - 10 or more s...,...,Woodford,231.0,Woodford,NaN,NaN,1.0,1.0,2.0,1.0,326.0
1,000252-01012017,01/01/2017,2017,11:58:44,11,Special Service,Special Service,Lift Release,Dwelling,Purpose Built Flats/Maisonettes - 4 to 9 storeys,...,Stratford,NaN,NaN,NaN,NaN,1.0,1.0,1.0,2.0,652.0
2,000127-01012017,01/01/2017,2017,03:57:50,3,False Alarm,AFA,NaN,Dwelling,Purpose Built Flats/Maisonettes - 4 to 9 storeys,...,West Norwood,206.0,West Norwood,230.0,West Norwood,1.0,2.0,2.0,1.0,326.0
3,000378-01012017,01/01/2017,2017,19:05:55,19,Fire,Primary Fire,NaN,Dwelling,Purpose Built Flats/Maisonettes - 4 to 9 storeys,...,Feltham,258.0,Feltham,279.0,Feltham,1.0,2.0,5.0,11.0,3586.0
4,000289-01012017,01/01/2017,2017,13:37:52,13,False Alarm,AFA,NaN,Dwelling,Self contained Sheltered Housing,...,Surbiton,490.0,Kingston,579.0,Kingston,1.0,2.0,2.0,1.0,326.0


In [6]:
#LFB database for inspection (next csv file - 2013-2016)
LFB2016df = pd.read_csv('LFBIncident2016.csv', engine='python')
LFB2016df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403985 entries, 0 to 403984
Data columns (total 31 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   IncidentNumber                          403985 non-null  object 
 1   DateOfCall                              403985 non-null  object 
 2   CalYear                                 403985 non-null  int64  
 3   TimeOfCall                              403985 non-null  object 
 4   HourOfCall                              403985 non-null  int64  
 5   IncidentGroup                           403984 non-null  object 
 6   StopCodeDescription                     403984 non-null  object 
 7   SpecialServiceType                      123086 non-null  object 
 8   PropertyCategory                        403984 non-null  object 
 9   PropertyType                            403984 non-null  object 
 10  AddressQualifier                        4039

In [7]:
LFB2016df.head()

,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,...,Easting_rounded,Northing_rounded,FRS,IncidentStationGround,FirstPumpArriving_AttendanceTime,FirstPumpArriving_DeployedFromStation,SecondPumpArriving_AttendanceTime,SecondPumpArriving_DeployedFromStation,NumStationsWithPumpsAttending,NumPumpsAttending
0,1131,01-Jan-13,2013,00:02:06,0,False Alarm,AFA,NaN,Other Residential,Boarding House/B&B for homeless/asylum seekers,...,525450,184850,London,West Hampstead,167.0,West Hampstead,NaN,NaN,1.0,1.0
1,4131,01-Jan-13,2013,00:02:09,0,False Alarm,AFA,NaN,Non Residential,Single shop,...,515450,185450,London,Northolt,236.0,Northolt,NaN,NaN,1.0,1.0
2,5131,01-Jan-13,2013,00:02:54,0,False Alarm,AFA,NaN,Non Residential,Other cultural venue,...,522450,178650,London,Hammersmith,218.0,Hammersmith,NaN,NaN,1.0,1.0
3,2131,01-Jan-13,2013,00:03:02,0,Fire,Secondary Fire,NaN,Outdoor Structure,Small refuse/rubbish container,...,527850,181050,London,Soho,426.0,Knightsbridge,NaN,NaN,1.0,1.0
4,3131,01-Jan-13,2013,00:03:03,0,False Alarm,AFA,NaN,Non Residential,Purpose built office,...,533350,180750,London,Whitechapel,346.0,Dowgate,NaN,NaN,2.0,2.0


In [9]:
#Keeping only the relevant columns from both databases (two years from earlier csv) and combining.

LFB2017df1 = LFB2017df[['DateOfCall',
 'CalYear',
 'HourOfCall',
 'IncidentGroup',
 'PropertyType',
 'SpecialServiceType',
 'IncidentStationGround']]

LFB2017df1['Datetime'] = pd.to_datetime(LFB2017df1.DateOfCall+' '+LFB2017df1.HourOfCall.astype(str), format='%d/%m/%Y %H')

LFB2016df1 = LFB2016df[['DateOfCall',
 'CalYear',
 'HourOfCall',
 'IncidentGroup',
 'PropertyType',
 'SpecialServiceType',
 'IncidentStationGround']]

LFB2016df1['Datetime'] = pd.to_datetime(LFB2016df1.DateOfCall+' '+LFB2016df1.HourOfCall.astype(str), format='%d-%b-%y %H')
LFB = pd.concat([LFB2017df1, LFB2016df1[(LFB2016df1['CalYear'] == 2015)|(LFB2016df1['CalYear'] == 2016)]], ignore_index=True)
LFB.info()

<ipython-input-9-41d326f2b3da>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LFB2017df1['Datetime'] = pd.to_datetime(LFB2017df1.DateOfCall+' '+LFB2017df1.HourOfCall.astype(str), format='%d/%m/%Y %H')
<ipython-input-9-41d326f2b3da>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LFB2016df1['Datetime'] = pd.to_datetime(LFB2016df1.DateOfCall+' '+LFB2016df1.HourOfCall.astype(str), format='%d-%b-%y %H')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 616797 entries, 0 to 616796
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   DateOfCall             616797 non-null  object        
 1   CalYear                616797 non-null  int64         
 2   HourOfCall             616797 non-null  int64         
 3   IncidentGroup          616796 non-null  object        
 4   PropertyType           616796 non-null  object        
 5   SpecialServiceType     192141 non-null  object        
 6   IncidentStationGround  616797 non-null  object        
 7   Datetime               616797 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 37.6+ MB


In [10]:
#A closer look at the propertytype column - will be used for classification modelling
print(LFB.PropertyType.nunique())
LFB.PropertyType.value_counts()[:30]

286


Purpose Built Flats/Maisonettes - 4 to 9 storeys         76862
House - single occupancy                                 71512
Purpose Built Flats/Maisonettes - Up to 3 storeys        57765
Car                                                      29738
Self contained Sheltered Housing                         28640
Purpose built office                                     22444
Converted Flat/Maisonettes - 3 or more storeys           22301
Purpose Built Flats/Maisonettes - 10 or more storeys     19479
Converted Flat/Maisonette - Up to 2 storeys              18023
Single shop                                              15469
Small refuse/rubbish container                           15093
Retirement/Old Persons Home                              12535
Domestic garden (vegetation not equipment)               10000
Road surface/pavement                                     9971
Loose refuse                                              8893
Multiple Vehicles                                      

In [11]:
# holiday marker
import holidays
hols = []
for ptr in holidays.England(years = [2016,2017,2018,2019,2020]).items(): 
    hols.append(ptr[0]) 

In [12]:
print(hols)

[datetime.date(2016, 1, 1), datetime.date(2016, 3, 25), datetime.date(2016, 3, 28), datetime.date(2016, 5, 2), datetime.date(2016, 5, 30), datetime.date(2016, 8, 29), datetime.date(2016, 12, 25), datetime.date(2016, 12, 27), datetime.date(2016, 12, 26), datetime.date(2017, 1, 1), datetime.date(2017, 1, 2), datetime.date(2017, 4, 14), datetime.date(2017, 4, 17), datetime.date(2017, 5, 1), datetime.date(2017, 5, 29), datetime.date(2017, 8, 28), datetime.date(2017, 12, 25), datetime.date(2017, 12, 26), datetime.date(2018, 1, 1), datetime.date(2018, 3, 30), datetime.date(2018, 4, 2), datetime.date(2018, 5, 7), datetime.date(2018, 5, 28), datetime.date(2018, 8, 27), datetime.date(2018, 12, 25), datetime.date(2018, 12, 26), datetime.date(2019, 1, 1), datetime.date(2019, 4, 19), datetime.date(2019, 4, 22), datetime.date(2019, 5, 6), datetime.date(2019, 5, 27), datetime.date(2019, 8, 26), datetime.date(2019, 12, 25), datetime.date(2019, 12, 26), datetime.date(2020, 1, 1), datetime.date(2020, 4

In [13]:
LFB['holiday'] = 0
LFB['holiday'] = [1 if i in hols else 0 for i in LFB.Datetime]

In [14]:
# Weekend marker
LFB['weekend'] = 0
LFB['weekend'] = [1 if i in [5,6] else 0 for i in LFB.Datetime.dt.weekday]

In [15]:
LFB.weekend.sum()

179392

In [16]:
# lockdown marker
series = pd.Series(['2020/03/23', '2020/06/15', '2020/11/05', '2020/12/02','2021/01/05'])
lockdates = pd.to_datetime(series)

LFB['lockdown'] = 0
LFB['lockdown'] = [1 if (((i >= lockdates[0])&(i < lockdates[1]))|((i >= lockdates[2]) 
                                    & (i < lockdates[3]))|((i >= lockdates[4]) )) else 0 for i in LFB.Datetime]

In [17]:
LFB.lockdown.sum()

28220

In [18]:
# Filling the special service null values.
LFB = LFB.fillna(0)

# Met Office Data #

## 2016 Weather ##

In [19]:
# Now for the weather - starting with the met office csv files (pre-stripped of some irrelevant header rows).
Weath2016w = pd.read_csv('uk-hourly-weather-obs2016noleg.csv', engine='python')
Weath2016w.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8785 entries, 0 to 8784
Columns: 104 entries, ob_time to drv_hr_sun_dur_q
dtypes: float64(98), object(6)
memory usage: 7.0+ MB


In [20]:
list(Weath2016w.columns)

['ob_time',
 'id',
 'id_type',
 'met_domain_name',
 'version_num',
 'src_id',
 'rec_st_ind',
 'wind_speed_unit_id',
 'src_opr_type',
 'wind_direction',
 'wind_speed',
 'prst_wx_id',
 'past_wx_id_1',
 'past_wx_id_2',
 'cld_ttl_amt_id',
 'low_cld_type_id',
 'med_cld_type_id',
 'hi_cld_type_id',
 'cld_base_amt_id',
 'cld_base_ht',
 'visibility',
 'msl_pressure',
 'cld_amt_id_1',
 'cloud_type_id_1',
 'cld_base_ht_id_1',
 'cld_amt_id_2',
 'cloud_type_id_2',
 'cld_base_ht_id_2',
 'cld_amt_id_3',
 'cloud_type_id_3',
 'cld_base_ht_id_3',
 'cld_amt_id_4',
 'cloud_type_id_4',
 'cld_base_ht_id_4',
 'vert_vsby',
 'air_temperature',
 'dewpoint',
 'wetb_temp',
 'rltv_hum',
 'stn_pres',
 'alt_pres',
 'ground_state_id',
 'q10mnt_mxgst_spd',
 'cavok_flag',
 'cs_hr_sun_dur',
 'wmo_hr_sun_dur',
 'snow_depth',
 'wind_direction_q',
 'wind_speed_q',
 'prst_wx_id_q',
 'past_wx_id_1_q',
 'past_wx_id_2_q',
 'cld_ttl_amt_id_q',
 'low_cld_type_id_q',
 'med_cld_type_id_q',
 'hi_cld_type_id_q',
 'cld_base_amt_id_q

In [21]:
Weath2016w1 = Weath2016w[['ob_time',           
   'wind_direction',   
    'wind_speed',       
    'cld_ttl_amt_id',   
    'visibility',       
    'air_temperature',  
    'rltv_hum',         
    'stn_pres',         
    'wmo_hr_sun_dur']]

In [22]:
Weath2016w1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8785 entries, 0 to 8784
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ob_time          8785 non-null   object 
 1   wind_direction   8775 non-null   float64
 2   wind_speed       8775 non-null   float64
 3   cld_ttl_amt_id   8781 non-null   float64
 4   visibility       8781 non-null   float64
 5   air_temperature  8782 non-null   float64
 6   rltv_hum         8782 non-null   float64
 7   stn_pres         8782 non-null   float64
 8   wmo_hr_sun_dur   8414 non-null   float64
dtypes: float64(8), object(1)
memory usage: 617.8+ KB


In [23]:
Weath2016w1.head()

,ob_time,wind_direction,wind_speed,cld_ttl_amt_id,visibility,air_temperature,rltv_hum,stn_pres,wmo_hr_sun_dur
0,01/01/2016 00:00,240.0,6.0,0.0,2000.0,3.8,89.8,1017.8,NaN
1,01/01/2016 01:00,230.0,5.0,0.0,1400.0,3.7,91.5,1018.5,0.0
2,01/01/2016 02:00,0.0,0.0,0.0,700.0,2.6,94.6,1018.8,0.0
3,01/01/2016 03:00,170.0,3.0,0.0,800.0,2.0,94.3,1019.1,0.0
4,01/01/2016 04:00,110.0,3.0,0.0,700.0,2.3,96.4,1019.6,0.0


In [24]:
Weath2016r = pd.read_csv('uk-hourly-rain-obs_2016noleg.csv', engine='python')
Weath2016r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9490 entries, 0 to 9489
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ob_end_time       9490 non-null   object 
 1   id                9489 non-null   float64
 2   id_type           9489 non-null   object 
 3   ob_hour_count     9489 non-null   float64
 4   version_num       9489 non-null   float64
 5   met_domain_name   9489 non-null   object 
 6   src_id            9489 non-null   float64
 7   rec_st_ind        9489 non-null   float64
 8   prcp_amt          9488 non-null   float64
 9   prcp_dur          0 non-null      float64
 10  prcp_amt_q        9489 non-null   float64
 11  prcp_dur_q        1 non-null      float64
 12  prcp_amt_j        0 non-null      float64
 13  meto_stmp_time    9489 non-null   object 
 14  midas_stmp_etime  9487 non-null   float64
dtypes: float64(11), object(4)
memory usage: 1.1+ MB


In [25]:
Weath2016r.head()

,ob_end_time,id,id_type,ob_hour_count,version_num,met_domain_name,src_id,rec_st_ind,prcp_amt,prcp_dur,prcp_amt_q,prcp_dur_q,prcp_amt_j,meto_stmp_time,midas_stmp_etime
0,01/01/2016 00:00,247539.0,RAIN,1.0,1.0,SREW,708.0,1011.0,0.0,NaN,1.0,NaN,NaN,31/12/2015 23:52,0.0
1,01/01/2016 01:00,247539.0,RAIN,1.0,1.0,SREW,708.0,1011.0,0.0,NaN,1.0,NaN,NaN,01/01/2016 01:34,0.0
2,01/01/2016 02:00,247539.0,RAIN,1.0,1.0,SREW,708.0,1011.0,0.0,NaN,1.0,NaN,NaN,01/01/2016 01:51,0.0
3,01/01/2016 03:00,247539.0,RAIN,1.0,1.0,SREW,708.0,1011.0,0.0,NaN,1.0,NaN,NaN,01/01/2016 02:51,0.0
4,01/01/2016 04:00,247539.0,RAIN,1.0,1.0,SREW,708.0,1011.0,0.0,NaN,1.0,NaN,NaN,01/01/2016 03:52,0.0


In [26]:
Weath2016r.ob_hour_count.value_counts()

1.0     8760
12.0     726
24.0       3
Name: ob_hour_count, dtype: int64

In [27]:
Weath2016r1 = Weath2016r[Weath2016r['ob_hour_count'] == 1]
Weath2016r1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8760 entries, 0 to 9488
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ob_end_time       8760 non-null   object 
 1   id                8760 non-null   float64
 2   id_type           8760 non-null   object 
 3   ob_hour_count     8760 non-null   float64
 4   version_num       8760 non-null   float64
 5   met_domain_name   8760 non-null   object 
 6   src_id            8760 non-null   float64
 7   rec_st_ind        8760 non-null   float64
 8   prcp_amt          8759 non-null   float64
 9   prcp_dur          0 non-null      float64
 10  prcp_amt_q        8760 non-null   float64
 11  prcp_dur_q        0 non-null      float64
 12  prcp_amt_j        0 non-null      float64
 13  meto_stmp_time    8760 non-null   object 
 14  midas_stmp_etime  8758 non-null   float64
dtypes: float64(11), object(4)
memory usage: 1.1+ MB


In [28]:
# Finding missing values (to be filled from meteostat if a large number of continuous missing, or backfilled for odd
# missing hours).
miss2016 = []
for i in Weath2016w1['ob_time']:
    if i not in list(Weath2016r1['ob_end_time']):
        miss2016.append(i)
miss2016

['18/11/2016 10:00',
 '18/11/2016 11:00',
 '18/11/2016 12:00',
 '18/11/2016 13:00',
 '18/11/2016 14:00',
 '18/11/2016 15:00',
 '18/11/2016 16:00',
 '18/11/2016 17:00',
 '18/11/2016 18:00',
 '18/11/2016 19:00',
 '18/11/2016 20:00',
 '18/11/2016 21:00',
 '18/11/2016 22:00',
 '18/11/2016 23:00',
 '19/11/2016 00:00',
 '19/11/2016 01:00',
 '19/11/2016 02:00',
 '19/11/2016 03:00',
 '19/11/2016 04:00',
 '19/11/2016 05:00',
 '19/11/2016 06:00',
 '19/11/2016 07:00',
 '19/11/2016 08:00',
 '19/11/2016 09:00',
 'end data']

In [29]:
Weath2016r1 = Weath2016r1.rename(columns={"ob_end_time": "ob_time"}, errors="raise")
df2=Weath2016r1[['ob_time','prcp_amt']]
        
Weath2016w1 = Weath2016w1.merge(df2,how = 'left', on = 'ob_time')
Weath2016w1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8785 entries, 0 to 8784
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ob_time          8785 non-null   object 
 1   wind_direction   8775 non-null   float64
 2   wind_speed       8775 non-null   float64
 3   cld_ttl_amt_id   8781 non-null   float64
 4   visibility       8781 non-null   float64
 5   air_temperature  8782 non-null   float64
 6   rltv_hum         8782 non-null   float64
 7   stn_pres         8782 non-null   float64
 8   wmo_hr_sun_dur   8414 non-null   float64
 9   prcp_amt         8759 non-null   float64
dtypes: float64(9), object(1)
memory usage: 755.0+ KB


In [30]:
Weath2016wind = pd.read_csv('uk-mean-wind-obs_2016noleg.csv', engine='python')
Weath2016wind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8782 entries, 0 to 8781
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ob_end_time        8782 non-null   object 
 1   id_type            8781 non-null   object 
 2   id                 8781 non-null   float64
 3   ob_hour_count      8781 non-null   float64
 4   met_domain_name    8781 non-null   object 
 5   version_num        8781 non-null   float64
 6   src_id             8781 non-null   float64
 7   rec_st_ind         8781 non-null   float64
 8   mean_wind_dir      8773 non-null   float64
 9   mean_wind_speed    8773 non-null   float64
 10  max_gust_dir       8781 non-null   float64
 11  max_gust_speed     8781 non-null   float64
 12  max_gust_ctime     8781 non-null   float64
 13  mean_wind_dir_q    8773 non-null   float64
 14  mean_wind_speed_q  8773 non-null   float64
 15  max_gust_dir_q     8781 non-null   float64
 16  max_gust_speed_q   8781 

In [31]:
Weath2016wind.head()

,ob_end_time,id_type,id,ob_hour_count,met_domain_name,version_num,src_id,rec_st_ind,mean_wind_dir,mean_wind_speed,...,mean_wind_speed_q,max_gust_dir_q,max_gust_speed_q,max_gust_ctime_q,mean_wind_dir_j,mean_wind_speed_j,max_gust_dir_j,max_gust_speed_j,meto_stmp_time,midas_stmp_etime
0,01/01/2016 00:00,WIND,511305.0,1.0,HCM,1.0,708.0,1011.0,260.0,8.0,...,6.0,6.0,6.0,6.0,NaN,NaN,NaN,NaN,31/12/2015 23:52,0.0
1,01/01/2016 01:00,WIND,511305.0,1.0,HCM,1.0,708.0,1011.0,230.0,5.0,...,6.0,6.0,6.0,6.0,NaN,NaN,NaN,NaN,01/01/2016 01:34,0.0
2,01/01/2016 02:00,WIND,511305.0,1.0,HCM,1.0,708.0,1011.0,220.0,2.0,...,6.0,6.0,6.0,6.0,NaN,NaN,NaN,NaN,01/01/2016 01:51,0.0
3,01/01/2016 03:00,WIND,511305.0,1.0,HCM,1.0,708.0,1011.0,170.0,3.0,...,6.0,6.0,6.0,6.0,NaN,NaN,NaN,NaN,01/01/2016 02:51,0.0
4,01/01/2016 04:00,WIND,511305.0,1.0,HCM,1.0,708.0,1011.0,150.0,4.0,...,6.0,6.0,6.0,6.0,NaN,NaN,NaN,NaN,01/01/2016 03:52,0.0


In [32]:
Weath2016wind1 = Weath2016wind[['ob_end_time','max_gust_speed']]
Weath2016wind1 = Weath2016wind1.rename(columns={"ob_end_time": "ob_time"}, errors="raise")
Weath2016w1 = Weath2016w1.merge(Weath2016wind1,how = 'left', on = 'ob_time')
Weath2016w1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8785 entries, 0 to 8784
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ob_time          8785 non-null   object 
 1   wind_direction   8775 non-null   float64
 2   wind_speed       8775 non-null   float64
 3   cld_ttl_amt_id   8781 non-null   float64
 4   visibility       8781 non-null   float64
 5   air_temperature  8782 non-null   float64
 6   rltv_hum         8782 non-null   float64
 7   stn_pres         8782 non-null   float64
 8   wmo_hr_sun_dur   8414 non-null   float64
 9   prcp_amt         8759 non-null   float64
 10  max_gust_speed   8781 non-null   float64
dtypes: float64(10), object(1)
memory usage: 823.6+ KB


In [33]:
# Evaluate missing values.
miss2016wind = []
for i in Weath2016w1['ob_time']:
    if i not in list(Weath2016wind1['ob_time']):
        miss2016wind.append(i)
miss2016wind

['22/03/2016 13:00', '20/06/2016 10:00', '20/06/2016 11:00']

In [34]:
# Last row is NaN
Weath2016w2 = Weath2016w1.drop(index = (len(Weath2016w1)-1))

In [35]:
Weath2016w2['ob_time'] = pd.to_datetime(Weath2016w2.ob_time)

In [36]:
Weath2016w2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8784 entries, 0 to 8783
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ob_time          8784 non-null   datetime64[ns]
 1   wind_direction   8775 non-null   float64       
 2   wind_speed       8775 non-null   float64       
 3   cld_ttl_amt_id   8781 non-null   float64       
 4   visibility       8781 non-null   float64       
 5   air_temperature  8782 non-null   float64       
 6   rltv_hum         8782 non-null   float64       
 7   stn_pres         8782 non-null   float64       
 8   wmo_hr_sun_dur   8414 non-null   float64       
 9   prcp_amt         8759 non-null   float64       
 10  max_gust_speed   8781 non-null   float64       
dtypes: datetime64[ns](1), float64(10)
memory usage: 823.5 KB


In [37]:
# Importing missing data
from meteostat import Stations, Daily, Hourly
from datetime import datetime
# Set time period
start = datetime(2016, 11, 18)
end = datetime(2016, 11, 19, 9, 59)

# Get hourly data
data = Hourly('03772', start, end)
data2016weathr = data.fetch()

for i in data2016weathr.index:
    Weath2016w2[['prcp_amt']].loc[i] = data2016weathr[['prcp']].loc[i]


C:\Users\andre\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [38]:
# Now for sunrise and sunset times (for 'islight' marker)
Sunrise2016 = pd.read_csv('sunrise2016.csv', engine='python')
Sunrise2016.columns

Index(['NOAA Solar Calculations - Change any of the highlighted cells to get solar position data for that location and time-of-day for a year.',
       'Unnamed: 1', 'Unnamed: 2', 'Date', 'Time (hrs past local midnight)',
       'Julian Day', 'Julian Century', 'Unnamed: 7',
       'Geom Mean Long Sun (deg)', 'Geom Mean Anom Sun (deg)',
       'Eccent Earth Orbit', 'Sun Eq of Ctr', 'Sun True Long (deg)',
       'Sun True Anom (deg)', 'Sun Rad Vector (AUs)', 'Sun App Long (deg)',
       'Mean Obliq Ecliptic (deg)', 'Obliq Corr (deg)', 'Sun Rt Ascen (deg)',
       'Sun Declin (deg)', 'var y', 'Eq of Time (minutes)', 'HA Sunrise (deg)',
       'Solar Noon (LST)', 'Sunrise Time (LST)', 'Sunset Time (LST)',
       'Sunlight Duration (minutes)', 'True Solar Time (min)',
       'Hour Angle (deg)', 'Solar Zenith Angle (deg)',
       'Solar Elevation Angle (deg)', 'Approx Atmospheric Refraction (deg)',
       'Solar Elevation corrected for atm refraction (deg)',
       'Solar Azimuth Angle (deg 

In [39]:
Sunrise20161 = Sunrise2016[['Date','Sunrise Time (LST)','Sunset Time (LST)']]
Sunrise20161.Date = pd.to_datetime(Sunrise20161.Date)
Weath2016w2['ob_time1'] = Weath2016w2.ob_time.dt.date
Sunrise20161['ob_time1'] = Sunrise20161['Date'].dt.date
Weath2016w2 = Weath2016w2.merge(Sunrise20161,how = 'left', on = 'ob_time1')
Weath2016w2.head()

C:\Users\andre\anaconda3\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<ipython-input-39-35b5af636fad>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Sunrise20161['ob_time1'] = Sunrise20161['Date'].dt.date


,ob_time,wind_direction,wind_speed,cld_ttl_amt_id,visibility,air_temperature,rltv_hum,stn_pres,wmo_hr_sun_dur,prcp_amt,max_gust_speed,ob_time1,Date,Sunrise Time (LST),Sunset Time (LST)
0,2016-01-01 00:00:00,240.0,6.0,0.0,2000.0,3.8,89.8,1017.8,NaN,0.0,14.0,2016-01-01,2016-01-01,8:05:39,16:00:59
1,2016-01-01 01:00:00,230.0,5.0,0.0,1400.0,3.7,91.5,1018.5,0.0,0.0,8.0,2016-01-01,2016-01-01,8:05:39,16:00:59
2,2016-01-01 02:00:00,0.0,0.0,0.0,700.0,2.6,94.6,1018.8,0.0,0.0,5.0,2016-01-01,2016-01-01,8:05:39,16:00:59
3,2016-01-01 03:00:00,170.0,3.0,0.0,800.0,2.0,94.3,1019.1,0.0,0.0,6.0,2016-01-01,2016-01-01,8:05:39,16:00:59
4,2016-01-01 04:00:00,110.0,3.0,0.0,700.0,2.3,96.4,1019.6,0.0,0.0,6.0,2016-01-01,2016-01-01,8:05:39,16:00:59


In [40]:
# 'islight' marker
Weath2016w2['sunrisehr']=Weath2016w2['Sunrise Time (LST)'].str.replace(r':\d\d:\d\d','').astype(int)
Weath2016w2['sunsethr']=Weath2016w2['Sunset Time (LST)'].str.replace(r':\d\d:\d\d','').astype(int)

Weath2016w2['islight'] = 0
Weath2016w2['islight'] = [1 if ((Weath2016w2['sunrisehr'][i] <= Weath2016w2['ob_time'][i].hour) 
                                and (Weath2016w2['ob_time'][i].hour < Weath2016w2['sunsethr'][i])) 
                          else 0 for i in Weath2016w2.index]
                           

In [ ]:
# Now for the UTC/GMT to summertime adjustment (Weather GMT only, LFB BST as well)
Daylight saving time 2016 in United Kingdom began at 01:00 on
Sunday, 27 March
and ended at 02:00 on
Sunday, 30 October

In [41]:
Weath2016w2[Weath2016w2['ob_time'] == '2016-10-30 02:00:00']

,ob_time,wind_direction,wind_speed,cld_ttl_amt_id,visibility,air_temperature,rltv_hum,stn_pres,wmo_hr_sun_dur,prcp_amt,max_gust_speed,ob_time1,Date,Sunrise Time (LST),Sunset Time (LST),sunrisehr,sunsethr,islight
7274,2016-10-30 02:00:00,90.0,2.0,0.0,20.0,8.4,98.5,1028.6,0.0,0.0,3.0,2016-10-30,2016-10-30,6:50:56,16:36:15,6,16,0


In [42]:
Weath2016w3 = pd.concat([Weath2016w2.head(7274), pd.DataFrame(Weath2016w2.iloc[7274]).T, Weath2016w2.tail(len(Weath2016w2) - (7274))])
Weath2016w3.iloc[[7273,7274,7275,7276]]

,ob_time,wind_direction,wind_speed,cld_ttl_amt_id,visibility,air_temperature,rltv_hum,stn_pres,wmo_hr_sun_dur,prcp_amt,max_gust_speed,ob_time1,Date,Sunrise Time (LST),Sunset Time (LST),sunrisehr,sunsethr,islight
7273,2016-10-30 01:00:00,50,2,1,70,9.1,98.6,1029,0,0,3,2016-10-30,2016-10-30,6:50:56,16:36:15,6,16,0
7274,2016-10-30 02:00:00,90,2,0,20,8.4,98.5,1028.6,0,0,3,2016-10-30,2016-10-30,6:50:56,16:36:15,6,16,0
7274,2016-10-30 02:00:00,90,2,0,20,8.4,98.5,1028.6,0,0,3,2016-10-30,2016-10-30,6:50:56,16:36:15,6,16,0
7275,2016-10-30 03:00:00,250,2,7,10,9.6,100,1028.3,0,0,3,2016-10-30,2016-10-30,6:50:56,16:36:15,6,16,0


In [43]:
Weath2016w3.reset_index(inplace=True)
Weath2016w3.drop('index', axis=1, inplace=True)
Weath2016w3.iloc[[7273,7274,7275,7276]]

,ob_time,wind_direction,wind_speed,cld_ttl_amt_id,visibility,air_temperature,rltv_hum,stn_pres,wmo_hr_sun_dur,prcp_amt,max_gust_speed,ob_time1,Date,Sunrise Time (LST),Sunset Time (LST),sunrisehr,sunsethr,islight
7273,2016-10-30 01:00:00,50,2,1,70,9.1,98.6,1029,0,0,3,2016-10-30,2016-10-30,6:50:56,16:36:15,6,16,0
7274,2016-10-30 02:00:00,90,2,0,20,8.4,98.5,1028.6,0,0,3,2016-10-30,2016-10-30,6:50:56,16:36:15,6,16,0
7275,2016-10-30 02:00:00,90,2,0,20,8.4,98.5,1028.6,0,0,3,2016-10-30,2016-10-30,6:50:56,16:36:15,6,16,0
7276,2016-10-30 03:00:00,250,2,7,10,9.6,100,1028.3,0,0,3,2016-10-30,2016-10-30,6:50:56,16:36:15,6,16,0


In [44]:
Weath2016w2[Weath2016w2['ob_time'] == '2016-03-27 01:00:00']

,ob_time,wind_direction,wind_speed,cld_ttl_amt_id,visibility,air_temperature,rltv_hum,stn_pres,wmo_hr_sun_dur,prcp_amt,max_gust_speed,ob_time1,Date,Sunrise Time (LST),Sunset Time (LST),sunrisehr,sunsethr,islight
2065,2016-03-27 01:00:00,210.0,11.0,8.0,3500.0,7.1,81.1,997.8,0.0,0.0,14.0,2016-03-27,2016-03-27,5:45:21,18:25:04,5,18,0


In [45]:
Weath2016w4 = Weath2016w3.copy()
for i in range(2066,7274):
    Weath2016w4['ob_time'][i+1] = Weath2016w3['ob_time'][i]
Weath2016w4.iloc[[7271,7272,7273,7274,7275,7276]]

<ipython-input-45-0613b37d9fd7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Weath2016w4['ob_time'][i+1] = Weath2016w3['ob_time'][i]


,ob_time,wind_direction,wind_speed,cld_ttl_amt_id,visibility,air_temperature,rltv_hum,stn_pres,wmo_hr_sun_dur,prcp_amt,max_gust_speed,ob_time1,Date,Sunrise Time (LST),Sunset Time (LST),sunrisehr,sunsethr,islight
7271,2016-10-29 22:00:00,80,2,0,210,9.7,93.3,1029.6,0,0,4,2016-10-29,2016-10-29,6:49:11,16:38:07,6,16,0
7272,2016-10-29 23:00:00,30,1,0,90,9.8,98.7,1029.5,NaN,0,3,2016-10-30,2016-10-30,6:50:56,16:36:15,6,16,0
7273,2016-10-30 00:00:00,50,2,1,70,9.1,98.6,1029,0,0,3,2016-10-30,2016-10-30,6:50:56,16:36:15,6,16,0
7274,2016-10-30 01:00:00,90,2,0,20,8.4,98.5,1028.6,0,0,3,2016-10-30,2016-10-30,6:50:56,16:36:15,6,16,0
7275,2016-10-30 02:00:00,90,2,0,20,8.4,98.5,1028.6,0,0,3,2016-10-30,2016-10-30,6:50:56,16:36:15,6,16,0
7276,2016-10-30 03:00:00,250,2,7,10,9.6,100,1028.3,0,0,3,2016-10-30,2016-10-30,6:50:56,16:36:15,6,16,0


In [46]:
Weath2016w4.iloc[[2065,2066,2067,2068]]

,ob_time,wind_direction,wind_speed,cld_ttl_amt_id,visibility,air_temperature,rltv_hum,stn_pres,wmo_hr_sun_dur,prcp_amt,max_gust_speed,ob_time1,Date,Sunrise Time (LST),Sunset Time (LST),sunrisehr,sunsethr,islight
2065,2016-03-27 01:00:00,210,11,8,3500,7.1,81.1,997.8,0,0,14,2016-03-27,2016-03-27,5:45:21,18:25:04,5,18,0
2066,2016-03-27 02:00:00,200,8,8,3500,6.3,84.9,997.9,0,0,13,2016-03-27,2016-03-27,5:45:21,18:25:04,5,18,0
2067,2016-03-27 02:00:00,200,5,4,2200,5.8,89.3,997.5,0,0,10,2016-03-27,2016-03-27,5:45:21,18:25:04,5,18,0
2068,2016-03-27 03:00:00,180,8,1,1800,5.8,90.8,997.7,0,0,12,2016-03-27,2016-03-27,5:45:21,18:25:04,5,18,0


In [47]:
Weath2016w5 = Weath2016w4.drop(2066)
Weath2016w5.iloc[[2065,2066,2067,2068]]

,ob_time,wind_direction,wind_speed,cld_ttl_amt_id,visibility,air_temperature,rltv_hum,stn_pres,wmo_hr_sun_dur,prcp_amt,max_gust_speed,ob_time1,Date,Sunrise Time (LST),Sunset Time (LST),sunrisehr,sunsethr,islight
2065,2016-03-27 01:00:00,210,11,8,3500,7.1,81.1,997.8,0,0,14,2016-03-27,2016-03-27,5:45:21,18:25:04,5,18,0
2067,2016-03-27 02:00:00,200,5,4,2200,5.8,89.3,997.5,0,0,10,2016-03-27,2016-03-27,5:45:21,18:25:04,5,18,0
2068,2016-03-27 03:00:00,180,8,1,1800,5.8,90.8,997.7,0,0,12,2016-03-27,2016-03-27,5:45:21,18:25:04,5,18,0
2069,2016-03-27 04:00:00,180,9,2,2200,5.8,90.8,997.7,0,0,13,2016-03-27,2016-03-27,5:45:21,18:25:04,5,18,1


In [48]:
Weath2016w5.reset_index(inplace=True)
Weath2016w5.drop('index', axis=1, inplace=True)
Weath2016w5.iloc[[2065,2066,2067,2068]]

,ob_time,wind_direction,wind_speed,cld_ttl_amt_id,visibility,air_temperature,rltv_hum,stn_pres,wmo_hr_sun_dur,prcp_amt,max_gust_speed,ob_time1,Date,Sunrise Time (LST),Sunset Time (LST),sunrisehr,sunsethr,islight
2065,2016-03-27 01:00:00,210,11,8,3500,7.1,81.1,997.8,0,0,14,2016-03-27,2016-03-27,5:45:21,18:25:04,5,18,0
2066,2016-03-27 02:00:00,200,5,4,2200,5.8,89.3,997.5,0,0,10,2016-03-27,2016-03-27,5:45:21,18:25:04,5,18,0
2067,2016-03-27 03:00:00,180,8,1,1800,5.8,90.8,997.7,0,0,12,2016-03-27,2016-03-27,5:45:21,18:25:04,5,18,0
2068,2016-03-27 04:00:00,180,9,2,2200,5.8,90.8,997.7,0,0,13,2016-03-27,2016-03-27,5:45:21,18:25:04,5,18,1


In [49]:
Weath2016w5.columns

Index(['ob_time', 'wind_direction', 'wind_speed', 'cld_ttl_amt_id',
       'visibility', 'air_temperature', 'rltv_hum', 'stn_pres',
       'wmo_hr_sun_dur', 'prcp_amt', 'max_gust_speed', 'ob_time1', 'Date',
       'Sunrise Time (LST)', 'Sunset Time (LST)', 'sunrisehr', 'sunsethr',
       'islight'],
      dtype='object')

In [50]:
Weath2016w6 = Weath2016w5.drop(columns = ['ob_time1','Date','Sunrise Time (LST)', 'Sunset Time (LST)', 'sunrisehr', 'sunsethr'])
Weath2016w6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ob_time          8784 non-null   datetime64[ns]
 1   wind_direction   8775 non-null   object        
 2   wind_speed       8775 non-null   object        
 3   cld_ttl_amt_id   8781 non-null   object        
 4   visibility       8781 non-null   object        
 5   air_temperature  8782 non-null   object        
 6   rltv_hum         8782 non-null   object        
 7   stn_pres         8782 non-null   object        
 8   wmo_hr_sun_dur   8414 non-null   object        
 9   prcp_amt         8759 non-null   object        
 10  max_gust_speed   8781 non-null   object        
 11  islight          8784 non-null   object        
dtypes: datetime64[ns](1), object(11)
memory usage: 823.6+ KB


In [51]:
# Filling empty values appropriately.
Weath2016w6.fillna(method = 'backfill', inplace = True)
Weath2016w6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ob_time          8784 non-null   datetime64[ns]
 1   wind_direction   8784 non-null   float64       
 2   wind_speed       8784 non-null   float64       
 3   cld_ttl_amt_id   8784 non-null   float64       
 4   visibility       8784 non-null   float64       
 5   air_temperature  8784 non-null   float64       
 6   rltv_hum         8784 non-null   float64       
 7   stn_pres         8784 non-null   float64       
 8   wmo_hr_sun_dur   8784 non-null   float64       
 9   prcp_amt         8784 non-null   float64       
 10  max_gust_speed   8784 non-null   float64       
 11  islight          8784 non-null   int64         
dtypes: datetime64[ns](1), float64(10), int64(1)
memory usage: 823.6 KB


## 2017 Weather ##

In [ ]:
# Now for 2017, 2018, 2019. Function considered, but too many individual areas to adjust.

In [52]:
Weath2017w = pd.read_csv('uk-hourly-weather-obs2017noleg.csv', engine='python')
Weath2017w1 = Weath2017w[['ob_time', 'wind_direction', 'wind_speed', 'cld_ttl_amt_id', 'visibility', 'air_temperature',  
    'rltv_hum', 'stn_pres', 'wmo_hr_sun_dur']]
Weath2017r = pd.read_csv('uk-hourly-rain-obs_2017noleg.csv', engine='python')
Weath2017r1 = Weath2017r[Weath2017r['ob_hour_count'] == 1]
miss2017 = []
for i in Weath2017w1['ob_time']:
    if i not in list(Weath2017r1['ob_end_time']):
        miss2017.append(i)
miss2017

['end data']

In [53]:
Weath2017r1 = Weath2017r1.rename(columns={"ob_end_time": "ob_time"}, errors="raise")
df3=Weath2017r1[['ob_time','prcp_amt']]
        
Weath2017w1 = Weath2017w1.merge(df3,how = 'left', on = 'ob_time')
Weath2017wind = pd.read_csv('uk-mean-wind-obs_2017noleg.csv', engine='python')
Weath2017wind1 = Weath2017wind[['ob_end_time','max_gust_speed']]
Weath2017wind1 = Weath2017wind1.rename(columns={"ob_end_time": "ob_time"}, errors="raise")
Weath2017w1 = Weath2017w1.merge(Weath2017wind1,how = 'left', on = 'ob_time')
miss2017wind = []
for i in Weath2017w1['ob_time']:
    if i not in list(Weath2017wind1['ob_time']):
        miss2017wind.append(i)
miss2017wind

['21/06/2017 12:00',
 '12/10/2017 11:00',
 '12/10/2017 12:00',
 '14/11/2017 13:00']

In [54]:
Weath2017w2 = Weath2017w1.drop(index = (len(Weath2017w1)-1))
Weath2017w2['ob_time'] = pd.to_datetime(Weath2017w2.ob_time)

In [55]:
# Importing missing data (if needed)
from meteostat import Stations, Daily, Hourly
from datetime import datetime
# Set time period
start = datetime(2016, 11, 18)
end = datetime(2016, 11, 19, 9, 59)

# Get hourly data
data = Hourly('03772', start, end)
data2016weathr = data.fetch()

for i in data2016weathr.index:
    Weath2016w2[['prcp_amt']].loc[i] = data2016weathr[['prcp']].loc[i]


C:\Users\andre\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [56]:
Sunrise2017 = pd.read_csv('sunrise2017.csv', engine='python')
Sunrise20171 = Sunrise2017[['Date','Sunrise Time (LST)','Sunset Time (LST)']]
Sunrise20171.Date = pd.to_datetime(Sunrise20171.Date)
Weath2017w2['ob_time1'] = Weath2017w2.ob_time.dt.date
Sunrise20171['ob_time1'] = Sunrise20171['Date'].dt.date
Weath2017w2 = Weath2017w2.merge(Sunrise20171,how = 'left', on = 'ob_time1')
Weath2017w2['sunrisehr']=Weath2017w2['Sunrise Time (LST)'].str.replace(r':\d\d:\d\d','').astype(int)
Weath2017w2['sunsethr']=Weath2017w2['Sunset Time (LST)'].str.replace(r':\d\d:\d\d','').astype(int)

Weath2017w2['islight'] = 0
Weath2017w2['islight'] = [1 if ((Weath2017w2['sunrisehr'][i] <= Weath2017w2['ob_time'][i].hour) 
                                and (Weath2017w2['ob_time'][i].hour < Weath2017w2['sunsethr'][i])) 
                          else 0 for i in Weath2017w2.index]

C:\Users\andre\anaconda3\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<ipython-input-56-0dded22c5f98>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Sunrise20171['ob_time1'] = Sunrise20171['Date'].dt.date


In [ ]:
Daylight saving time 2017 in United Kingdom began at 01:00 on
Sunday, 26 March
and ended at 02:00 on
Sunday, 29 October

In [57]:
Weath2017w2[Weath2017w2['ob_time'] == '2017-10-29 02:00:00']

,ob_time,wind_direction,wind_speed,cld_ttl_amt_id,visibility,air_temperature,rltv_hum,stn_pres,wmo_hr_sun_dur,prcp_amt,max_gust_speed,ob_time1,Date,Sunrise Time (LST),Sunset Time (LST),sunrisehr,sunsethr,islight
7225,2017-10-29 02:00:00,260.0,13.0,8.0,5000.0,13.8,83.0,1015.3,0.0,0.0,19.0,2017-10-29,2017-10-29,6:48:45,16:38:35,6,16,0


In [58]:
Weath2017w3 = pd.concat([Weath2017w2.head(7225), pd.DataFrame(Weath2017w2.iloc[7225]).T, Weath2017w2.tail(len(Weath2017w2) - (7225))])
Weath2017w3.reset_index(inplace=True)
Weath2017w3.drop('index', axis=1, inplace=True)
Weath2017w2[Weath2017w2['ob_time'] == '2017-03-26 01:00:00']

,ob_time,wind_direction,wind_speed,cld_ttl_amt_id,visibility,air_temperature,rltv_hum,stn_pres,wmo_hr_sun_dur,prcp_amt,max_gust_speed,ob_time1,Date,Sunrise Time (LST),Sunset Time (LST),sunrisehr,sunsethr,islight
2017,2017-03-26 01:00:00,50.0,10.0,0.0,5000.0,6.2,48.0,1022.1,0.0,0.0,16.0,2017-03-26,2017-03-26,5:48:11,18:22:59,5,18,0


In [59]:
Weath2017w4 = Weath2017w3.copy()
for i in range(2018,7225):
    Weath2017w4['ob_time'][i+1] = Weath2017w3['ob_time'][i]
Weath2017w5 = Weath2017w4.drop(2018)

<ipython-input-59-7c82028e2e88>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Weath2017w4['ob_time'][i+1] = Weath2017w3['ob_time'][i]


In [60]:
Weath2017w5.reset_index(inplace=True)
Weath2017w5.drop('index', axis=1, inplace=True)
Weath2017w5.iloc[[2016,2017,2018,2019]]

,ob_time,wind_direction,wind_speed,cld_ttl_amt_id,visibility,air_temperature,rltv_hum,stn_pres,wmo_hr_sun_dur,prcp_amt,max_gust_speed,ob_time1,Date,Sunrise Time (LST),Sunset Time (LST),sunrisehr,sunsethr,islight
2016,2017-03-26 00:00:00,60,6,0,5000,6.7,40.8,1022.8,NaN,0,16,2017-03-26,2017-03-26,5:48:11,18:22:59,5,18,0
2017,2017-03-26 01:00:00,50,10,0,5000,6.2,48,1022.1,0,0,16,2017-03-26,2017-03-26,5:48:11,18:22:59,5,18,0
2018,2017-03-26 02:00:00,40,5,0,3500,5.5,67.3,1021.6,0,0,10,2017-03-26,2017-03-26,5:48:11,18:22:59,5,18,0
2019,2017-03-26 03:00:00,50,7,0,3000,5.6,74.9,1021.5,0,0,12,2017-03-26,2017-03-26,5:48:11,18:22:59,5,18,0


In [61]:
Weath2017w6 = Weath2017w5.drop(columns = ['ob_time1','Date','Sunrise Time (LST)', 'Sunset Time (LST)', 'sunrisehr', 'sunsethr'])
Weath2017w6.fillna(method = 'backfill', inplace = True)
Weath2017w6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8759 entries, 0 to 8758
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ob_time          8759 non-null   datetime64[ns]
 1   wind_direction   8759 non-null   float64       
 2   wind_speed       8759 non-null   float64       
 3   cld_ttl_amt_id   8759 non-null   float64       
 4   visibility       8759 non-null   float64       
 5   air_temperature  8759 non-null   float64       
 6   rltv_hum         8759 non-null   float64       
 7   stn_pres         8759 non-null   float64       
 8   wmo_hr_sun_dur   8759 non-null   float64       
 9   prcp_amt         8759 non-null   float64       
 10  max_gust_speed   8759 non-null   float64       
 11  islight          8759 non-null   int64         
dtypes: datetime64[ns](1), float64(10), int64(1)
memory usage: 821.3 KB


## 2018 Weather ##

In [62]:
Weath2018w = pd.read_csv('uk-hourly-weather-obs2018noleg.csv', engine='python')
Weath2018w1 = Weath2018w[['ob_time', 'wind_direction', 'wind_speed', 'cld_ttl_amt_id', 'visibility', 'air_temperature',  
    'rltv_hum', 'stn_pres', 'wmo_hr_sun_dur']]
Weath2018r = pd.read_csv('uk-hourly-rain-obs_2018noleg.csv', engine='python')
Weath2018r1 = Weath2018r[Weath2018r['ob_hour_count'] == 1]
miss2018 = []
for i in Weath2018w1['ob_time']:
    if i not in list(Weath2018r1['ob_end_time']):
        miss2018.append(i)
miss2018

['22/01/2018 15:00',
 '22/01/2018 16:00',
 '25/02/2018 10:00',
 '25/02/2018 11:00',
 '25/02/2018 12:00',
 '25/02/2018 13:00',
 '25/02/2018 14:00',
 '25/02/2018 15:00',
 '25/02/2018 16:00',
 '25/02/2018 17:00',
 '25/02/2018 18:00',
 '25/02/2018 19:00',
 '25/02/2018 20:00',
 '25/02/2018 21:00',
 '25/02/2018 22:00',
 '25/02/2018 23:00',
 '26/02/2018 00:00',
 '26/02/2018 01:00',
 '26/02/2018 02:00',
 '26/02/2018 03:00',
 '26/02/2018 04:00',
 '26/02/2018 05:00',
 '26/02/2018 06:00',
 '26/02/2018 07:00',
 '26/02/2018 08:00',
 '26/02/2018 09:00',
 '26/02/2018 10:00',
 '26/02/2018 11:00',
 '26/02/2018 12:00',
 '26/02/2018 13:00',
 '26/02/2018 14:00',
 '26/02/2018 15:00',
 '26/02/2018 16:00',
 '26/02/2018 17:00',
 '26/02/2018 18:00',
 '26/02/2018 19:00',
 '26/02/2018 20:00',
 '26/02/2018 21:00',
 '26/02/2018 22:00',
 '26/02/2018 23:00',
 '27/02/2018 00:00',
 '27/02/2018 01:00',
 '27/02/2018 02:00',
 '27/02/2018 03:00',
 '27/02/2018 04:00',
 '27/02/2018 05:00',
 '27/02/2018 06:00',
 '27/02/2018 

In [63]:
Weath2018r1 = Weath2018r1.rename(columns={"ob_end_time": "ob_time"}, errors="raise")
df4=Weath2018r1[['ob_time','prcp_amt']]
        
Weath2018w1 = Weath2018w1.merge(df4,how = 'left', on = 'ob_time')
Weath2018wind = pd.read_csv('uk-mean-wind-obs_2018noleg.csv', engine='python')
Weath2018wind1 = Weath2018wind[['ob_end_time','max_gust_speed']]
Weath2018wind1 = Weath2018wind1.rename(columns={"ob_end_time": "ob_time"}, errors="raise")
Weath2018w1 = Weath2018w1.merge(Weath2018wind1,how = 'left', on = 'ob_time')
miss2018wind = []
for i in Weath2018w1['ob_time']:
    if i not in list(Weath2018wind1['ob_time']):
        miss2018wind.append(i)
miss2018wind

['22/01/2018 15:00',
 '22/01/2018 16:00',
 '08/02/2018 13:00',
 '15/02/2018 15:00',
 '29/06/2018 14:00',
 '29/06/2018 15:00']

In [64]:
Weath2018w2 = Weath2018w1.drop(index = (len(Weath2018w1)-1))
Weath2018w2['ob_time'] = pd.to_datetime(Weath2018w2.ob_time)

In [65]:
# Importing missing data (if needed)
from meteostat import Stations, Daily, Hourly
from datetime import datetime
# Set time period
start = datetime(2018, 2, 25)
end = datetime(2018, 3, 20, 9, 59)

# Get hourly data
data = Hourly('03772', start, end)
data2018weathr = data.fetch()

for i in data2018weathr.index:
    Weath2018w2[['prcp_amt']].loc[i] = data2018weathr[['prcp']].loc[i]

C:\Users\andre\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [66]:
Sunrise2018 = pd.read_csv('sunrise2018.csv', engine='python')
Sunrise20181 = Sunrise2018[['Date','Sunrise Time (LST)','Sunset Time (LST)']]
Sunrise20181.Date = pd.to_datetime(Sunrise20181.Date)
Weath2018w2['ob_time1'] = Weath2018w2.ob_time.dt.date
Sunrise20181['ob_time1'] = Sunrise20181['Date'].dt.date
Weath2018w2 = Weath2018w2.merge(Sunrise20181,how = 'left', on = 'ob_time1')
Weath2018w2['sunrisehr']=Weath2018w2['Sunrise Time (LST)'].str.replace(r':\d\d:\d\d','').astype(int)
Weath2018w2['sunsethr']=Weath2018w2['Sunset Time (LST)'].str.replace(r':\d\d:\d\d','').astype(int)

Weath2018w2['islight'] = 0
Weath2018w2['islight'] = [1 if ((Weath2018w2['sunrisehr'][i] <= Weath2018w2['ob_time'][i].hour) 
                                and (Weath2018w2['ob_time'][i].hour < Weath2018w2['sunsethr'][i])) 
                          else 0 for i in Weath2018w2.index]

C:\Users\andre\anaconda3\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<ipython-input-66-99c49d08017c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Sunrise20181['ob_time1'] = Sunrise20181['Date'].dt.date


In [ ]:
Daylight saving time 2018 in United Kingdom began at 01:00 on
Sunday, 25 March
and ended at 02:00 on
Sunday, 28 October

In [67]:
Weath2018w2[Weath2018w2['ob_time'] == '2018-10-28 02:00:00']

,ob_time,wind_direction,wind_speed,cld_ttl_amt_id,visibility,air_temperature,rltv_hum,stn_pres,wmo_hr_sun_dur,prcp_amt,max_gust_speed,ob_time1,Date,Sunrise Time (LST),Sunset Time (LST),sunrisehr,sunsethr,islight
7191,2018-10-28 02:00:00,360.0,8.0,0.0,3500.0,4.0,88.2,1013.6,0.0,0.0,17.0,2018-10-28,2018-10-28,6:46:34,16:40:56,6,16,0


In [68]:
Weath2018w3 = pd.concat([Weath2018w2.head(7191), pd.DataFrame(Weath2018w2.iloc[7191]).T, Weath2018w2.tail(len(Weath2018w2) - (7191))])
Weath2018w3.reset_index(inplace=True)
Weath2018w3.drop('index', axis=1, inplace=True)
Weath2018w2[Weath2018w2['ob_time'] == '2018-03-25 01:00:00']

,ob_time,wind_direction,wind_speed,cld_ttl_amt_id,visibility,air_temperature,rltv_hum,stn_pres,wmo_hr_sun_dur,prcp_amt,max_gust_speed,ob_time1,Date,Sunrise Time (LST),Sunset Time (LST),sunrisehr,sunsethr,islight
1990,2018-03-25 01:00:00,10.0,3.0,8.0,380.0,7.6,91.4,1002.8,0.0,0.0,7.0,2018-03-25,2018-03-25,5:51:02,18:20:54,5,18,0


In [69]:
Weath2018w4 = Weath2018w3.copy()
for i in range(1991,7191):
    Weath2018w4['ob_time'][i+1] = Weath2018w3['ob_time'][i]
Weath2018w5 = Weath2018w4.drop(1991)

<ipython-input-69-be11a55ad7d3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Weath2018w4['ob_time'][i+1] = Weath2018w3['ob_time'][i]


In [70]:
Weath2018w5.reset_index(inplace=True)
Weath2018w5.drop('index', axis=1, inplace=True)
Weath2018w5.iloc[[1990,1990,1992]]

,ob_time,wind_direction,wind_speed,cld_ttl_amt_id,visibility,air_temperature,rltv_hum,stn_pres,wmo_hr_sun_dur,prcp_amt,max_gust_speed,ob_time1,Date,Sunrise Time (LST),Sunset Time (LST),sunrisehr,sunsethr,islight
1990,2018-03-25 01:00:00,10,3,8,380,7.6,91.4,1002.8,0,0,7,2018-03-25,2018-03-25,5:51:02,18:20:54,5,18,0
1990,2018-03-25 01:00:00,10,3,8,380,7.6,91.4,1002.8,0,0,7,2018-03-25,2018-03-25,5:51:02,18:20:54,5,18,0
1992,2018-03-25 03:00:00,10,4,8,280,7.3,92.7,1003.9,0,0,8,2018-03-25,2018-03-25,5:51:02,18:20:54,5,18,0


In [71]:
Weath2018w6 = Weath2018w5.drop(columns = ['ob_time1','Date','Sunrise Time (LST)', 'Sunset Time (LST)', 'sunrisehr', 'sunsethr'])
Weath2018w6.fillna(method = 'backfill', inplace = True)
Weath2018w6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8749 entries, 0 to 8748
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ob_time          8749 non-null   datetime64[ns]
 1   wind_direction   8749 non-null   float64       
 2   wind_speed       8749 non-null   float64       
 3   cld_ttl_amt_id   8749 non-null   float64       
 4   visibility       8749 non-null   float64       
 5   air_temperature  8749 non-null   float64       
 6   rltv_hum         8749 non-null   float64       
 7   stn_pres         8749 non-null   float64       
 8   wmo_hr_sun_dur   8749 non-null   float64       
 9   prcp_amt         8749 non-null   float64       
 10  max_gust_speed   8749 non-null   float64       
 11  islight          8749 non-null   int64         
dtypes: datetime64[ns](1), float64(10), int64(1)
memory usage: 820.3 KB


## 2019 Weather ##

In [72]:
Weath2019w = pd.read_csv('uk-hourly-weather-obs2019noleg.csv', engine='python')
Weath2019w1 = Weath2019w[['ob_time', 'wind_direction', 'wind_speed', 'cld_ttl_amt_id', 'visibility', 'air_temperature',  
    'rltv_hum', 'stn_pres', 'wmo_hr_sun_dur']]
Weath2019r = pd.read_csv('uk-hourly-rain-obs_2019noleg.csv', engine='python')
Weath2019r1 = Weath2019r[Weath2019r['ob_hour_count'] == 1]
miss2019 = []
for i in Weath2019w1['ob_time']:
    if i not in list(Weath2019r1['ob_end_time']):
        miss2019.append(i)
miss2019

['24/04/2019 10:00',
 '24/04/2019 11:00',
 '24/04/2019 12:00',
 '24/04/2019 13:00',
 '24/04/2019 14:00',
 '24/04/2019 15:00',
 '24/04/2019 16:00',
 '24/04/2019 17:00',
 '24/04/2019 18:00',
 '24/04/2019 19:00',
 '24/04/2019 20:00',
 '24/04/2019 21:00',
 '24/04/2019 22:00',
 '24/04/2019 23:00',
 '25/04/2019 00:00',
 '25/04/2019 01:00',
 '25/04/2019 02:00',
 '25/04/2019 03:00',
 '25/04/2019 04:00',
 '25/04/2019 05:00',
 '25/04/2019 06:00',
 '25/04/2019 07:00',
 '25/04/2019 08:00',
 '25/04/2019 09:00',
 'end data']

In [73]:
Weath2019r1 = Weath2019r1.rename(columns={"ob_end_time": "ob_time"}, errors="raise")
df5=Weath2019r1[['ob_time','prcp_amt']]
        
Weath2019w1 = Weath2019w1.merge(df5,how = 'left', on = 'ob_time')
Weath2019wind = pd.read_csv('uk-mean-wind-obs_2019noleg.csv', engine='python')
Weath2019wind1 = Weath2019wind[['ob_end_time','max_gust_speed']]
Weath2019wind1 = Weath2019wind1.rename(columns={"ob_end_time": "ob_time"}, errors="raise")
Weath2019w1 = Weath2019w1.merge(Weath2019wind1,how = 'left', on = 'ob_time')
miss2019wind = []
for i in Weath2019w1['ob_time']:
    if i not in list(Weath2019wind1['ob_time']):
        miss2019wind.append(i)
miss2019wind

['17/04/2019 22:00',
 '25/04/2019 01:00',
 '08/05/2019 22:00',
 '08/05/2019 23:00',
 '09/05/2019 00:00',
 '09/05/2019 01:00',
 '09/05/2019 02:00',
 '15/05/2019 22:00',
 '15/05/2019 23:00',
 '16/05/2019 02:00',
 '24/05/2019 12:00',
 '24/05/2019 14:00',
 '24/05/2019 15:00',
 '03/07/2019 16:00',
 '11/10/2019 07:00',
 '11/10/2019 08:00']

In [74]:
Weath2019w2 = Weath2019w1.drop(index = (len(Weath2019w1)-1))
Weath2019w2['ob_time'] = pd.to_datetime(Weath2019w2.ob_time)

In [75]:
# Importing missing data (if needed)
from meteostat import Stations, Daily, Hourly
from datetime import datetime
# Set time period
start = datetime(2019, 4, 24)
end = datetime(2019, 4, 25, 9, 59)

# Get hourly data
data = Hourly('03772', start, end)
data2019weathr = data.fetch()

for i in data2019weathr.index:
    Weath2019w2[['prcp_amt']].loc[i] = data2019weathr[['prcp']].loc[i]

C:\Users\andre\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [76]:
Sunrise2019 = pd.read_csv('sunrise2019.csv', engine='python')
Sunrise20191 = Sunrise2019[['Date','Sunrise Time (LST)','Sunset Time (LST)']]
Sunrise20191.Date = pd.to_datetime(Sunrise20191.Date)
Weath2019w2['ob_time1'] = Weath2019w2.ob_time.dt.date
Sunrise20191['ob_time1'] = Sunrise20191['Date'].dt.date
Weath2019w2 = Weath2019w2.merge(Sunrise20191,how = 'left', on = 'ob_time1')
Weath2019w2['sunrisehr']=Weath2019w2['Sunrise Time (LST)'].str.replace(r':\d\d:\d\d','').astype(int)
Weath2019w2['sunsethr']=Weath2019w2['Sunset Time (LST)'].str.replace(r':\d\d:\d\d','').astype(int)

Weath2019w2['islight'] = 0
Weath2019w2['islight'] = [1 if ((Weath2019w2['sunrisehr'][i] <= Weath2019w2['ob_time'][i].hour) 
                                and (Weath2019w2['ob_time'][i].hour < Weath2019w2['sunsethr'][i])) 
                          else 0 for i in Weath2019w2.index]

C:\Users\andre\anaconda3\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<ipython-input-76-1455fba9065f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Sunrise20191['ob_time1'] = Sunrise20191['Date'].dt.date


In [ ]:
Daylight saving time 2019 in United Kingdom began at 01:00 on
Sunday, 31 March
and ended at 02:00 on
Sunday, 27 October

In [77]:
Weath2019w2[Weath2019w2['ob_time'] == '2019-10-27 02:00:00']

,ob_time,wind_direction,wind_speed,cld_ttl_amt_id,visibility,air_temperature,rltv_hum,stn_pres,wmo_hr_sun_dur,prcp_amt,max_gust_speed,ob_time1,Date,Sunrise Time (LST),Sunset Time (LST),sunrisehr,sunsethr,islight
7175,2019-10-27 02:00:00,290.0,7.0,3.0,1600.0,6.1,90.7,1016.4,0.0,0.0,10.0,2019-10-27,2019-10-27,6:44:23,16:43:19,6,16,0


In [78]:
Weath2019w3 = pd.concat([Weath2019w2.head(7175), pd.DataFrame(Weath2019w2.iloc[7175]).T, Weath2019w2.tail(len(Weath2019w2) - (7175))])
Weath2019w3.reset_index(inplace=True)
Weath2019w3.drop('index', axis=1, inplace=True)
Weath2019w2[Weath2019w2['ob_time'] == '2019-03-31 01:00:00']

,ob_time,wind_direction,wind_speed,cld_ttl_amt_id,visibility,air_temperature,rltv_hum,stn_pres,wmo_hr_sun_dur,prcp_amt,max_gust_speed,ob_time1,Date,Sunrise Time (LST),Sunset Time (LST),sunrisehr,sunsethr,islight
2137,2019-03-31 01:00:00,30.0,6.0,2.0,600.0,10.5,70.9,1018.9,0.0,0.0,10.0,2019-03-31,2019-03-31,5:37:54,18:30:33,5,18,0


In [79]:
Weath2019w4 = Weath2019w3.copy()
for i in range(2138,7175):
    Weath2019w4['ob_time'][i+1] = Weath2019w3['ob_time'][i]
Weath2019w5 = Weath2019w4.drop(2138)

<ipython-input-79-0a61460f5bfe>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Weath2019w4['ob_time'][i+1] = Weath2019w3['ob_time'][i]


In [80]:
Weath2019w5.reset_index(inplace=True)
Weath2019w5.drop('index', axis=1, inplace=True)
Weath2019w5.iloc[[2136,2137,2138]]

,ob_time,wind_direction,wind_speed,cld_ttl_amt_id,visibility,air_temperature,rltv_hum,stn_pres,wmo_hr_sun_dur,prcp_amt,max_gust_speed,ob_time1,Date,Sunrise Time (LST),Sunset Time (LST),sunrisehr,sunsethr,islight
2136,2019-03-31 00:00:00,10,5,6,700,11.3,65.8,1018.7,NaN,0,14,2019-03-31,2019-03-31,5:37:54,18:30:33,5,18,0
2137,2019-03-31 01:00:00,30,6,2,600,10.5,70.9,1018.9,0,0,10,2019-03-31,2019-03-31,5:37:54,18:30:33,5,18,0
2138,2019-03-31 02:00:00,40,6,5,280,9.6,80,1019.3,0,0,11,2019-03-31,2019-03-31,5:37:54,18:30:33,5,18,0


In [81]:
Weath2019w6 = Weath2019w5.drop(columns = ['ob_time1','Date','Sunrise Time (LST)', 'Sunset Time (LST)', 'sunrisehr', 'sunsethr'])
Weath2019w6.fillna(method = 'backfill', inplace = True)
Weath2019w6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8757 entries, 0 to 8756
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ob_time          8757 non-null   datetime64[ns]
 1   wind_direction   8757 non-null   float64       
 2   wind_speed       8757 non-null   float64       
 3   cld_ttl_amt_id   8757 non-null   float64       
 4   visibility       8757 non-null   float64       
 5   air_temperature  8757 non-null   float64       
 6   rltv_hum         8757 non-null   float64       
 7   stn_pres         8757 non-null   float64       
 8   wmo_hr_sun_dur   8757 non-null   float64       
 9   prcp_amt         8757 non-null   float64       
 10  max_gust_speed   8757 non-null   float64       
 11  islight          8757 non-null   int64         
dtypes: datetime64[ns](1), float64(10), int64(1)
memory usage: 821.1 KB


## 2020 Weather ##

In [82]:
from meteostat import Hourly, Daily
from meteostat import Stations
from datetime import datetime
# Set time period
start = datetime(2020, 1, 1)
end = datetime(2020, 12, 31, 23, 59)

# Get hourly data
data = Hourly('03772', start, end)
data2020weath = data.fetch()

# Print DataFrame
data2020weath.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8784 entries, 2020-01-01 00:00:00 to 2020-12-31 23:00:00
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   temp    8784 non-null   float64
 1   dwpt    8784 non-null   float64
 2   rhum    8784 non-null   float64
 3   prcp    8784 non-null   float64
 4   snow    0 non-null      float64
 5   wdir    8784 non-null   float64
 6   wspd    8784 non-null   float64
 7   wpgt    8784 non-null   float64
 8   pres    8784 non-null   float64
 9   tsun    0 non-null      float64
 10  coco    8784 non-null   float64
dtypes: float64(11)
memory usage: 823.5 KB


In [ ]:
# table for converting weather codes to cloud cover and sun (to get closer to met office columns)

Coco: Cloud     Sun
1       0        1
2	2
3	6
4	8
5	8
6	8
7	8



Code	Weather Condition
1	Clear
2	Fair
3	Cloudy
4	Overcast
5	Fog
6	Freezing Fog
7	Light Rain
8	Rain
9	Heavy Rain
10	Freezing Rain
11	Heavy Freezing Rain
12	Sleet
13	Heavy Sleet
14	Light Snowfall
15	Snowfall
16	Heavy Snowfall
17	Rain Shower
18	Heavy Rain Shower
19	Sleet Shower
20	Heavy Sleet Shower
21	Snow Shower
22	Heavy Snow Shower
23	Lightning
24	Hail
25	Thunderstorm
26	Heavy Thunderstorm
27	Storm

Visibility measures

VP (Very Poor)	Less than 1,000 metres
P (Poor)	Between 1,001 and 4,000 metres
M (Medium)	Between 4,001 and 10,000 metres
G (Good)	Between 10,001 and 20,000 metres
VG (Very Good)	Between 20,001 and 40,000 metres
E (Excellent)	Greater than 40,000 metres

In [83]:
data2020weath['sun'] = 0
data2020weath['visibility'] = 0
data2020weath['cld_ttl_amt_id'] = 0
for i in range(len(data2020weath.coco)):
    if data2020weath.coco[i] == 1.0:
        data2020weath['visibility'][i] = 5000
        data2020weath['cld_ttl_amt_id'][i] = 0
        data2020weath['sun'][i] = 1.0
    elif data2020weath.coco[i] == 2.0:
        data2020weath['visibility'][i] = 4000
        data2020weath['cld_ttl_amt_id'][i] = 2
        data2020weath['sun'][i] = 0.5
    elif data2020weath.coco[i] == 3.0:
        data2020weath['visibility'][i] = 4000
        data2020weath['cld_ttl_amt_id'][i] = 6
    elif data2020weath.coco[i] == 4.0:
        data2020weath['visibility'][i] = 4000
        data2020weath['cld_ttl_amt_id'][i] = 8
    elif data2020weath.coco[i] == 5.0:
        data2020weath['visibility'][i] = 100
        data2020weath['cld_ttl_amt_id'][i] = 8
    elif data2020weath.coco[i] == 6.0:
        data2020weath['visibility'][i] = 100
        data2020weath['cld_ttl_amt_id'][i] = 8
    elif data2020weath.coco[i] == 7.0:
        data2020weath['visibility'][i] = 2000
        data2020weath['cld_ttl_amt_id'][i] = 8
    elif data2020weath.coco[i] == 8.0:
        data2020weath['visibility'][i] = 2000
        data2020weath['cld_ttl_amt_id'][i] = 8
    elif data2020weath.coco[i] == 17.0:
        data2020weath['visibility'][i] = 2000
        data2020weath['cld_ttl_amt_id'][i] = 8
    else: 
        data2020weath['visibility'][i] = 1000
        data2020weath['cld_ttl_amt_id'][i] = 8
            

<ipython-input-83-4a66c51af48f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2020weath['visibility'][i] = 100
<ipython-input-83-4a66c51af48f>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2020weath['cld_ttl_amt_id'][i] = 8
<ipython-input-83-4a66c51af48f>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2020weath['visibility'][i] = 4000
<ipython-input-83-4a66c51af48f>:18: SettingWithCopyWarning: 
A value is trying 

In [84]:
# Correct units for windspeed
data2020weath['wind_speed'] = data2020weath['wspd']*0.539957

In [85]:
# Renaming columns
data2020weath = data2020weath.rename(columns={"wdir": "wind_direction",
                                              "temp":"air_temperature", "rhum":"rltv_hum","pres":"stn_pres", 
                                              "sun":"wmo_hr_sun_dur", "prcp":"prcp_amt",
                                              "wpgt":"max_gust_speed"}, errors="raise")

In [86]:
# Dropping irrelevant columns
data2020weatha = data2020weath.drop(columns = ['dwpt','snow','wspd','tsun','coco'])

In [87]:
data2020weath1 = data2020weatha.reset_index()
data2020weath1 = data2020weath1.rename(columns={"time": "ob_time"}, errors="raise")

In [88]:
data2020weath1.ob_time

0      2020-01-01 00:00:00
1      2020-01-01 01:00:00
2      2020-01-01 02:00:00
3      2020-01-01 03:00:00
4      2020-01-01 04:00:00
               ...        
8779   2020-12-31 19:00:00
8780   2020-12-31 20:00:00
8781   2020-12-31 21:00:00
8782   2020-12-31 22:00:00
8783   2020-12-31 23:00:00
Name: ob_time, Length: 8784, dtype: datetime64[ns]

In [89]:
Sunrise2020 = pd.read_csv('sunrise2020.csv', engine='python')
Sunrise20201 = Sunrise2020[['Date','Sunrise Time (LST)','Sunset Time (LST)']]
Sunrise20201.Date = pd.to_datetime(Sunrise20201.Date)
data2020weath1['ob_time1'] = data2020weath1.ob_time.dt.date
Sunrise20201['ob_time1'] = Sunrise20201['Date'].dt.date
Weath2020w2 = data2020weath1.merge(Sunrise20201,how = 'left', on = 'ob_time1')
Weath2020w2['sunrisehr']=Weath2020w2['Sunrise Time (LST)'].str.replace(r':\d\d:\d\d','').astype(int)
Weath2020w2['sunsethr']=Weath2020w2['Sunset Time (LST)'].str.replace(r':\d\d:\d\d','').astype(int)

Weath2020w2['islight'] = 0
Weath2020w2['islight'] = [1 if ((Weath2020w2['sunrisehr'][i] <= Weath2020w2['ob_time'][i].hour) 
                                and (Weath2020w2['ob_time'][i].hour < Weath2020w2['sunsethr'][i])) 
                          else 0 for i in Weath2020w2.index]

C:\Users\andre\anaconda3\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<ipython-input-89-2d89038cf033>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Sunrise20201['ob_time1'] = Sunrise20201['Date'].dt.date


In [90]:
Weath2020w5 = Weath2020w2.drop(columns = ['ob_time1','Date','Sunrise Time (LST)', 'Sunset Time (LST)', 'sunrisehr', 'sunsethr'])
Weath2020w6 = Weath2020w5.astype({'air_temperature': 'float64' ,
 'rltv_hum': 'float64',
 'prcp_amt': 'float64',
 'wind_direction': 'float64',
 'max_gust_speed': 'float64',
 'stn_pres': 'float64',
 'wmo_hr_sun_dur': 'float64',
 'visibility': 'float64',
 'cld_ttl_amt_id': 'float64',
 'wind_speed': 'float64',
 'islight' : 'int64'})
Weath2020w6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8784 entries, 0 to 8783
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ob_time          8784 non-null   datetime64[ns]
 1   air_temperature  8784 non-null   float64       
 2   rltv_hum         8784 non-null   float64       
 3   prcp_amt         8784 non-null   float64       
 4   wind_direction   8784 non-null   float64       
 5   max_gust_speed   8784 non-null   float64       
 6   stn_pres         8784 non-null   float64       
 7   wmo_hr_sun_dur   8784 non-null   float64       
 8   visibility       8784 non-null   float64       
 9   cld_ttl_amt_id   8784 non-null   float64       
 10  wind_speed       8784 non-null   float64       
 11  islight          8784 non-null   int64         
dtypes: datetime64[ns](1), float64(10), int64(1)
memory usage: 1.2 MB


# Merging Data #

In [91]:
LFB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 616797 entries, 0 to 616796
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   DateOfCall             616797 non-null  object        
 1   CalYear                616797 non-null  int64         
 2   HourOfCall             616797 non-null  int64         
 3   IncidentGroup          616797 non-null  object        
 4   PropertyType           616797 non-null  object        
 5   SpecialServiceType     616797 non-null  object        
 6   IncidentStationGround  616797 non-null  object        
 7   Datetime               616797 non-null  datetime64[ns]
 8   holiday                616797 non-null  int64         
 9   weekend                616797 non-null  int64         
 10  lockdown               616797 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(5)
memory usage: 51.8+ MB


In [92]:
LFB = LFB.rename(columns={"Datetime": "ob_time"}, errors="raise")

In [93]:
weather = pd.concat([Weath2016w6, Weath2017w6, Weath2018w6, Weath2019w6, Weath2020w6], ignore_index=True, sort=True)
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43833 entries, 0 to 43832
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   air_temperature  43833 non-null  float64       
 1   cld_ttl_amt_id   43833 non-null  float64       
 2   islight          43833 non-null  int64         
 3   max_gust_speed   43833 non-null  float64       
 4   ob_time          43833 non-null  datetime64[ns]
 5   prcp_amt         43833 non-null  float64       
 6   rltv_hum         43833 non-null  float64       
 7   stn_pres         43833 non-null  float64       
 8   visibility       43833 non-null  float64       
 9   wind_direction   43833 non-null  float64       
 10  wind_speed       43833 non-null  float64       
 11  wmo_hr_sun_dur   43833 non-null  float64       
dtypes: datetime64[ns](1), float64(10), int64(1)
memory usage: 4.0 MB


In [94]:
LFBwkg = LFB.merge(weather, on = 'ob_time')
LFBwkg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 518227 entries, 0 to 518226
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   DateOfCall             518227 non-null  object        
 1   CalYear                518227 non-null  int64         
 2   HourOfCall             518227 non-null  int64         
 3   IncidentGroup          518227 non-null  object        
 4   PropertyType           518227 non-null  object        
 5   SpecialServiceType     518227 non-null  object        
 6   IncidentStationGround  518227 non-null  object        
 7   ob_time                518227 non-null  datetime64[ns]
 8   holiday                518227 non-null  int64         
 9   weekend                518227 non-null  int64         
 10  lockdown               518227 non-null  int64         
 11  air_temperature        518227 non-null  float64       
 12  cld_ttl_amt_id         518227 non-null  floa

In [95]:
LFB2015 = LFB[LFB['CalYear'] == 2015].reset_index(drop = True)
LFB2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98353 entries, 0 to 98352
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   DateOfCall             98353 non-null  object        
 1   CalYear                98353 non-null  int64         
 2   HourOfCall             98353 non-null  int64         
 3   IncidentGroup          98353 non-null  object        
 4   PropertyType           98353 non-null  object        
 5   SpecialServiceType     98353 non-null  object        
 6   IncidentStationGround  98353 non-null  object        
 7   ob_time                98353 non-null  datetime64[ns]
 8   holiday                98353 non-null  int64         
 9   weekend                98353 non-null  int64         
 10  lockdown               98353 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(5)
memory usage: 8.3+ MB


In [96]:
LFBwkg.to_csv('Assets/LFBwkg.csv')
LFB2015.to_csv('Assets/LFB2015.csv')